# Converting Tensorflow model into a Core ML model 

## 1. Install coremltools 

In [0]:
!pip install --upgrade tfcoreml
!pip install --upgrade coremltools

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 3.4MB 7.7MB/s 
Requirement already up-to-date: coremltools in /usr/local/lib/python3.6/dist-packages (3.1)


In [0]:
from __future__ import print_function
import  os, sys, zipfile
from os.path import dirname
import numpy as np
import tensorflow as tf
from tensorflow.core.framework import graph_pb2

## 3. Download the model and class label package
First we'll download the frozen TensorFlow model and unzip it.


url = The URL address of the frozen file

dir_path = local directory 

In [0]:

# Download the model and class label package
def download_file_and_unzip(url, dir_path='.'):

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    k = url.rfind('/')
    fname = url[k+1:]
    fpath = os.path.join(dir_path, fname)

    if not os.path.exists(fpath):
        if sys.version_info[0] < 3:
            import urllib
            urllib.urlretrieve(url, fpath)
        else:
            import urllib.request
            urllib.request.urlretrieve(url, fpath)

    zip_ref = zipfile.ZipFile(fpath, 'r')
    zip_ref.extractall(dir_path)
    zip_ref.close()

inception_v3_url = 'https://storage.googleapis.com/download.tensorflow.org/models/inception_dec_2015.zip'
download_file_and_unzip(inception_v3_url);

##4. Load the TF graph definition

In [0]:
tf_model_path = './tensorflow_inception_graph.pb'
with open(tf_model_path, 'rb') as f:
    serialized = f.read()
tf.reset_default_graph()
original_gdef = tf.GraphDef()
original_gdef.ParseFromString(serialized)

# For demonstration purpose we show the first 15 ops the TF model
with tf.Graph().as_default() as g:
    tf.import_graph_def(original_gdef, name='')
    ops = g.get_operations()
    for i in range(15):
        print('op id {} : op name: {}, op type: "{}"'.format(str(i),ops[i].name, ops[i].type));

# This Inception model uses DecodeJpeg op to read from JPEG images
# encoded as string Tensors. You can visualize it with TensorBoard,
# but we're omitting it here. For deployment we need to remove the
# JPEG decoder and related ops, and replace them with a placeholder
# where we can feed image data in.

op id 0 : op name: DecodeJpeg/contents, op type: "Const"
op id 1 : op name: DecodeJpeg, op type: "DecodeJpeg"
op id 2 : op name: Cast, op type: "Cast"
op id 3 : op name: ExpandDims/dim, op type: "Const"
op id 4 : op name: ExpandDims, op type: "ExpandDims"
op id 5 : op name: ResizeBilinear/size, op type: "Const"
op id 6 : op name: ResizeBilinear, op type: "ResizeBilinear"
op id 7 : op name: Sub/y, op type: "Const"
op id 8 : op name: Sub, op type: "Sub"
op id 9 : op name: Mul/y, op type: "Const"
op id 10 : op name: Mul, op type: "Mul"
op id 11 : op name: conv/conv2d_params, op type: "Const"
op id 12 : op name: conv/Conv2D, op type: "Conv2D"
op id 13 : op name: conv/batchnorm/beta, op type: "Const"
op id 14 : op name: conv/batchnorm/gamma, op type: "Const"


## 5. Strip the decoder and prepocessing part
In this model, the actual op that feeds pre-processed image into the network is 'Mul'. The op that generates probabilities per class is 'softmax/logits'. To figure out what are inputs/outputs for your own model you can use use TensorFlow's summarize_graph or TensorBoard visualization tool for your own models.

In [0]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile
input_node_names = ['Mul']
output_node_names = ['softmax/logits']
gdef = strip_unused_lib.strip_unused(
        input_graph_def = original_gdef,
        input_node_names = input_node_names,
        output_node_names = output_node_names,
        placeholder_type_enum = dtypes.float32.as_datatype_enum)
# Save it to an output file
frozen_model_file = './inception_v3.pb'
with gfile.GFile(frozen_model_file, "wb") as f:
    f.write(gdef.SerializeToString())

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


## 6.Convert to CoreML

In [0]:

# Now we have a TF model ready to be converted to CoreML
import tfcoreml
# Supply a dictionary of input tensors' name and shape (with 
# batch axis)
input_tensor_shapes = {"Mul:0":[1,299,299,3]} # batch size is 1
# Output CoreML model path
coreml_model_file = './inception_v3.mlmodel'
# The TF model's ouput tensor name
output_tensor_names = ['softmax/logits:0']

# Call the converter.
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_file,
        input_name_shape_dict=input_tensor_shapes,
        output_feature_names=output_tensor_names,
        image_input_names = ['Mul:0'],
        red_bias = -1,
        green_bias = -1,
        blue_bias = -1,
        image_scale = 2.0/255.0)

# MLModel saved at location: ./inception_v3.mlmodel


Loading the TF graph...



Graph Loaded.
Now finding ops in the TF graph that can be dropped for inference
Done.
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob Mul:0
1/993: Analysing op name: softmax/biases ( type:  Const )
2/993: Analysing op name: softmax/weights ( type:  Const )
3/993: Analysing op name: pool_3/_reshape/shape ( type:  Const )
4/993: Analysing op name: mixed_10/join/concat_dim ( type:  Const )
5/993: Analysing op name: mixed_10/tower_2/conv/batchnorm/moving_variance ( type:  Const )
6/993: Analysing op name: mixed_10/tower_2/conv/batchnorm/moving_mean ( type:  Const )
7/993: Analysing op name: mixed_10/tower_2/conv/batchnorm/gamma ( type:  Const )
8/993: Analysing op name: mixed_10/tower_2/conv/batchnorm/beta ( type:  Const )
9/993: Analysing op name: mixed_10/tower_2/conv/conv2d_params ( type:  Const )
10/993: Analysing op name: mixed_10/tower_1/mixed/conv_1/batchnorm/moving_variance ( type:  Const )
11/993: Analysing op name: mixed_10/tower_